In [74]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import missingno as msno
from sklearn.metrics import confusion_matrix

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
# Функция для анализа данных в столбце
def column_analisys(clmn):
    print(clmn.describe())
    clmn.hist()
    IQR = clmn.quantile(0.75) - clmn.quantile(0.25)
    perc25 = clmn.quantile(0.25)
    perc75 = clmn.quantile(0.75)

    print(
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

    clmn.loc[clmn.between(perc25 - 1.5*IQR, perc75 + 1.5*IQR)
             ].hist(bins=10, range=(clmn.min(), clmn.max()), label='IQR')

    plt.legend()
    return

In [76]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR + '/train.csv')
df_test = pd.read_csv(DATA_DIR + '/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [77]:
sample_submission.shape

In [78]:
df_test.shape

In [79]:
df_train.info()

In [80]:
df_train.head(5)

In [81]:
df_test.info()

In [82]:
sample_submission.head(5)

In [83]:
sample_submission.info()

In [84]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1  # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
# в тесте у нас нет значения default, мы его должны предсказать, по этому пока просто заполняем нулями
df_test['default'] = 0

data = df_test.append(df_train, sort=False).reset_index(
    drop=True)  # объединяем

In [85]:
data.nunique(dropna=False)

In [86]:
# Показываем наличие пропусков
msno.bar(data)

In [87]:
num_cols = ['age', 'score_bki', 'decline_app_cnt',
            'score_bki', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna',
            'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

In [88]:
data['default'].value_counts(ascending=True).plot(kind='barh')

In [89]:
data['default'].value_counts(ascending=True)

In [90]:
# убираем id клиента, как не несущую информации
data.drop(['client_id'], axis=1, inplace=True)

In [91]:
# Преобразуем дату

data['app_date'] = pd.to_datetime(data['app_date'])

In [92]:
# Выделяем год, месяц и день
data['app_year'] = data['app_date'].apply(lambda x: x.year)
data['app_month'] = data['app_date'].apply(lambda x: x.month)
data['app_day'] = data['app_date'].apply(lambda x: x.day)

#data['app_weekday']= data['app_date'].apply(lambda x: x.day_of_week)

In [93]:
# Удаляем столбец с датой
data.drop(['app_date'], axis=1, inplace=True)

In [94]:
data[['app_year', 'app_month', 'app_day']]

In [95]:
# Заполняем пропуски
data['education'] = data['education'].fillna('SCH')

In [96]:
# Анализируем данные "возраст"
column_analisys(data['age'])

In [97]:
# Нормализуем данные "возраст"
data['age'] = data['age'].apply(lambda x: np.log(x))

In [98]:
column_analisys(data['age'])

In [99]:
column_analisys(data['score_bki'])

In [100]:
# Убираем выбросы
data['score_bki'].loc[data['score_bki'] > -
                      0.5290698762499992] = -0.5290698762499992
data['score_bki'].loc[data['score_bki'] < -
                      3.299251946250001] = -3.299251946250001

In [101]:
column_analisys(data['score_bki'])

In [102]:
column_analisys(data['decline_app_cnt'])

In [103]:
data['decline_app_cnt'].value_counts()

In [104]:
# Создаем новый столбец с категориями количества отказов
data['decline_app_cnt_cat'] = data['decline_app_cnt'].apply(
    lambda x: x == 0 and 0 or x == 1 and 1 or x == 2 and 2 or x == 3 and 3 or x <= 606 and 4)

In [105]:
data['decline_app_cnt_cat'].value_counts()

In [106]:
# Создаем новый столбец с бинарной категорией отказов

data['decline_app_cnt_bin'] = data['decline_app_cnt'].apply(
    lambda x: x != 0 and 1 or x == 0 and 0)

In [107]:
data['decline_app_cnt_bin'].unique()

In [108]:
column_analisys(data['bki_request_cnt'])

In [109]:
data['bki_request_cnt'].loc[data['bki_request_cnt'] > 7.5].count()

In [110]:
# Удаляем выбросы
data['bki_request_cnt'].loc[data['bki_request_cnt'] > 7.5] = 7.5

In [111]:
column_analisys(data['bki_request_cnt'])

In [112]:
data['bki_request_cnt'].loc[data['bki_request_cnt'].isna()] = 1.872487

In [113]:
column_analisys(data['bki_request_cnt'])

In [114]:
data['bki_request_cnt'].isna()

In [115]:
msno.bar(data)

In [116]:
column_analisys(data['income'])

In [117]:
data['income'] = data['income'].apply(lambda x: np.log(x))

In [118]:
column_analisys(data['income'])

In [119]:
# Создем новые данные на основе уже имеющихся
data['age_2'] = data['age']**2
data['income_2'] = data['income']**2
data['age_income'] = data['age']+data['income']/1000
data['age_income_mult'] = data['age']*data['income']/1000
# data['score_bki_sigm']=data['score_bki']*10000
# data['age_income_2']=data['age_income']**2
# data['decline_bki_cnt']=data['bki_request_cnt']*data['decline_app_cnt']
data['decline_bki_cnt'] = data['decline_app_cnt']*1000
data['score_bki_age'] = data['score_bki']*data['age']
data['score_bki_incom'] = data['score_bki']*data['income']/1000
data['score_bki_incom_age'] = data['score_bki']*data['age']*data['income']/1000
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income', 'age_2', 'income_2', 'age_income', 'app_year', 'app_month',
            'app_day', 'score_bki_age', 'score_bki_incom', 'score_bki_incom_age', 'age_income_mult', 'decline_bki_cnt', 'decline_app_cnt_bin']

In [120]:
data = pd.get_dummies(data, columns=['education'], dummy_na=True)
data = pd.get_dummies(data, columns=['decline_app_cnt_cat'], dummy_na=True)
# Попытка преобразовать "education" не в бинарныый, а мултикатегорийный признак, неуд-но
#from sklearn.preprocessing import LabelEncoder
#from sklearn import preprocessing
#from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.preprocessing import OrdinalEncoder

#mlb = LabelEncoder()


#le = preprocessing.LabelEncoder()
# le.fit(data['education'])

# le.classes_

# mapped_education=le.transform(data['education'])


# print(dict(enumerate(le.classes_)))
# убедимся в преобразовании
# data['education']=mapped_education

In [121]:
sns.heatmap(data[num_cols].corr().abs(), vmin=0, vmax=1)

In [122]:
from sklearn.feature_selection import f_classif
imp_num = pd.Series(f_classif(data[num_cols], data['default'])[
                    0], index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

In [123]:
data[['score_bki', 'decline_app_cnt', 'bki_request_cnt']]

In [124]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for column in bin_cols:
    data[column] = le.fit_transform(data[column])

columns = ['first_time', 'sna', 'work_address',
           'home_address', 'region_rating']

for column in columns:
    data[column] = le.fit_transform(data[column])

In [125]:
data.info()

In [126]:
data

In [127]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [128]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
from sklearn.model_selection import train_test_split

# выделим 15% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=42, shuffle=True)

In [129]:
# Вариант разделения выборки, результат неудовлетворительный
#import pandas as geek
#ones = y[y == 1.0]
# big_class=train_data[train_data['default']==0]
# small_class=train_data[train_data['default']==1]
#big_class_n = big_class.iloc[:len(ones), ]
# new_sample=pd.concat([big_class_n,small_class])

#X = new_sample[list(set(new_sample.columns) - set(['default']))]
#y = new_sample['default']

In [130]:
# X

In [131]:
# X.shape

In [132]:
# y

In [133]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [134]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [135]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# запускаем GridSearch на небольшом кол-ве итераций max_iter=50 и с достаточно большой дельтой останова tol1e-3
# чтобы получить оптимальные параметры модели в первом приближении
model = LogisticRegression()

iter_ = 50
epsilon_stop = 1e-3

param_grid = [
    {'penalty': ['l1'],
     'solver': ['liblinear', 'lbfgs'],
     'class_weight':['none', 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['l2'],
     'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
     'class_weight':['none', 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
    {'penalty': ['none'],
     'solver': ['newton-cg', 'lbfgs', 'sag', 'saga'],
     'class_weight':['none', 'balanced'],
     'multi_class': ['auto', 'ovr'],
     'max_iter':[iter_],
     'tol':[epsilon_stop]},
]

# model ваша модель логистической регрессии
gridsearch = GridSearchCV(model, param_grid, scoring='f1', n_jobs=-1, cv=5)
gridsearch.fit(X_train, y_train)
model = gridsearch.best_estimator_

# печатаем параметры
best_parameters = model.get_params()
for param_name in sorted(best_parameters.keys()):
    print('\t%s: %r' % (param_name, best_parameters[param_name]))

In [136]:
# Импортируем необходимые библиотеки:
# инструмент для создания и обучения модели
from sklearn.linear_model import LogisticRegression
from sklearn import metrics  # инструменты для оценки точности модели

In [137]:
logreg = LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                            fit_intercept=True,
                            intercept_scaling=1,
                            l1_ratio=None,
                            max_iter=100,
                            multi_class='auto',
                            n_jobs=None,
                            penalty='l2',
                            random_state=None,
                            solver='newton-cg',
                            tol=0.001,
                            verbose=0,
                            warm_start=False)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

In [138]:
# Метрики для оченки качества модели до размещения на Kaggle
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f'acc = {acc:.3f} F1-score = {f1:.5f}')

In [139]:
# Confusion matrix


confusion_matrix(y_test, y_pred)

In [140]:
from sklearn.metrics import classification_report
classification_report = classification_report(y_test, y_pred)
print(classification_report)

In [141]:
# если качество нас устраивает, обучаем финальную модель на всех обучающи данных
logreg_final = LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                                  fit_intercept=True,
                                  intercept_scaling=1,
                                  l1_ratio=None,
                                  max_iter=100,
                                  multi_class='auto',
                                  n_jobs=None,
                                  penalty='l2',
                                  random_state=None,
                                  solver='newton-cg',
                                  tol=0.001,
                                  verbose=0,
                                  warm_start=False)
logreg_final.fit(X, y)

In [142]:
predict_submission = logreg_final.predict(test_data)

In [143]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [144]:
sample_submission.describe()

In [145]:
!kaggle competitions submit - c sf-scoring - f ssubmission.csv - m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'